* 定义item数据结构
* 从json中取出

In [ ]:
import time

from qwikidata.entity import WikidataItem
from qwikidata.json_dump import WikidataJsonDump
from qwikidata.utils import dump_entities_to_json
import json

# P_OCCUPATION = "P106"
# Q_POLITICIAN = "Q82955"

# def has_occupation_politician(item: WikidataItem, truthy: bool = True) -> bool:
#     """Return True if the Wikidata Item has occupation politician."""
#     if truthy:
#         claim_group = item.get_truthy_claim_group(P_OCCUPATION)
#     else:
#         claim_group = item.get_claim_group(P_OCCUPATION)

#     occupation_qids = [
#         claim.mainsnak.datavalue.value["id"]
#         for claim in claim_group
#         if claim.mainsnak.snaktype == "value"
#     ]
#     return Q_POLITICIAN in occupation_qids


# create an instance of WikidataJsonDump
wjd_dump_path = "wikidata\wikidata-20231215-all.json.bz2"
wjd = WikidataJsonDump(wjd_dump_path)

# create an iterable of WikidataItem representing politicians
politicians = []
t1 = time.time()
for ii, entity_dict in enumerate(wjd):
    print(entity_dict)
    if entity_dict["type"] == "item":
        entity = WikidataItem(entity_dict)
        # if has_occupation_politician(entity):
        politicians.append(entity)
    if ii % 1000 == 0 and ii > 0:
        t2 = time.time()
        dt = t2 - t1
        print(
            "found {} politicians among {} entities [entities/s: {:.2f}]".format(
                len(politicians), ii, ii / dt
            )
        )
    if ii == 1:
        break


# write the iterable of WikidataItem to disk as JSON
out_fname = "wikidata\wikidata-20231215-all.json"
dump_entities_to_json(politicians, out_fname)

In [ ]:
from qwikidata.json_dump import WikidataJsonDump
from qwikidata.entity import WikidataItem
from qwikidata.utils import dump_entities_to_json
from tqdm import tqdm
import os
import json


wjd_dump_path = "wikidata\wikidata-20231215-all.json.bz2"
wjd = WikidataJsonDump(wjd_dump_path)

output_file_folder = "wikidata/20231215-en-label-description"
if not os.path.exists(output_file_folder):
    os.makedirs(output_file_folder)
batch_size = 1000000
wikidata_list_en=[]
for ii, entity_dict in tqdm(enumerate(wjd)):
    if entity_dict["type"] == "item":
        entity = WikidataItem(entity_dict)
        wikidata_list_en.append({
            'id':entity.entity_id,
            'language': 'en',
            'label': entity.get_label(),
            'description': entity.get_description()})
    if ii % batch_size == 0 and ii > 0:
        json_file= os.path.join(output_file_folder, 'wikidata-20231215-en-label-description-'+str(ii)+'.json')
        with open(json_file, 'w') as outfile:
            json.dump(wikidata_list_en, outfile)
        wikidata_list_en=[]
        print("saved {} entities".format(ii))

# 将最后一批数据保存到json文件中
json_file= os.path.join(output_file_folder, 'wikidata-20231215-en-label-description-'+str(ii)+'.json')
with open(json_file, 'w') as outfile:
    json.dump(wikidata_list_en, outfile)

In [8]:
# 将数据插入es
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from qwikidata.json_dump import WikidataJsonDump
from qwikidata.entity import WikidataItem
from qwikidata.utils import dump_entities_to_json
import json
import os
import time
from tqdm import tqdm

es = Elasticsearch(hosts="http://localhost:9200")
index_name = "wikidata-latest"
es.indices.create(index=index_name,ignore=400)

wjd_dump_path = "wikidata\wikidata-20231215-all.json.bz2"
wjd = WikidataJsonDump(wjd_dump_path)

batch_size=10000
item_list=[]

for ii, entity_dict in tqdm(enumerate(wjd)):
    if entity_dict["type"] == "item":
        entity = WikidataItem(entity_dict)
        item={
            '_id': entity.entity_id,
            '_index': index_name,
            'entity_id':entity.entity_id,
            'label':entity.get_label(),
            'description':entity.get_description(),
            'enwiki_title':entity.get_enwiki_title(),
        }
        item_list.append(item)
        if len(item_list) > batch_size:
            helpers.bulk(client=es, actions=item_list)
            item_list=[]
            print(f"index {ii} has been inserted.")


c:\Users\lqw\miniconda3\envs\simcse\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  
0it [00:00, ?it/s]

10108it [00:27, 94.60it/s] 

index 10000 has been inserted.


20107it [00:47, 136.47it/s]

index 20001 has been inserted.


...

106652421it [25:19:38, 614.40it/s] 

index 106652072 has been inserted.


106662569it [25:19:42, 821.15it/s] 

index 106662074 has been inserted.


106669271it [25:19:44, 1169.82it/s]
